In [2]:
import pandas as pd

## Load Excel File

In [3]:
df=pd.read_excel("Meta_data.xlsx")
df.columns = df.columns.str.strip().str.lower()
print(df.columns)
display(df)

Index(['table_names', 'column_names', 'primary_key'], dtype='object')


,table_names,column_names,primary_key
0,TBL_AAPR_AC_PREPRO,AAPR_LOCK_TOKEN,0
1,TBL_AAPR_AC_PREPRO,AAPR_ROW_ID,0
2,TBL_AAPR_AC_PREPRO,AAPR_SOURCE_IND,0
3,TBL_AAPR_AC_PREPRO,AAPR_WEIGHT,0
4,TBL_ABEF_WORK_TABLE,ABEF_ACC_CNT,0
...,...,...,...
18176,TBL_ZPCD_ZIP_DATA,ZPCD_ZIP_PFX,1
18177,TBL_ZPRB_ZIP_DATA,ZPRB_EFF_DT,1
18178,TBL_ZPRB_ZIP_DATA,ZPRB_LOCK_TOKEN,0
18179,TBL_ZPRB_ZIP_DATA,ZPRB_TERM_DT,0


In [4]:
target_tables = [
    'tbl_clcl_claim', 'tbl_clcb_cl_cob', 'tbl_clov_overpay', 'tbl_clhi_proc', 'tbl_mcre_relat_ent',
    'tbl_acrh_red_hist', 'tbl_clmf_mult_func', 'tbl_mctr_cd_trans', 'tbl_clis_its_subsc',
    'tbl_excd_expl_cd', 'tbl_ckck_check', 'tbl_clmd_diag_desc', 'tbl_ipcd_proc_cd', 'tbl_mecb_cob',
    'tbl_cdml_cl_line', 'tbl_pypy_payor', 'tbl_mepr_prim_prov', 'tbl_plds_plan_desc',
    'tbl_clck_clm_check', 'tbl_lobd_line_bus', 'tbl_pdpd_product', 'tbl_idcd_diag_cd',
    'tbl_prad_address', 'tbl_clhi_proc_desc', 'tbl_clst_status', 'tbl_clip_its_patnt',
    'tbl_clpp_its_prov', 'tbl_clhi_pivot', 'tbl_cdpp_li_its_pr', 'tbl_clmi_misc', 'tbl_clor_cl_ovr',
    'tbl_sgsg_sub_group', 'tbl_clmd_diag', 'tbl_ckpy_payee_sum', 'tbl_meme_member',
    'tbl_sbsb_subsc', 'tbl_mepe_prcs_elig', 'tbl_clhp_hosp', 'tbl_cddl_cl_line'
]

In [13]:
full_sql=""
for table in target_tables:
    table_df = df[df['table_names'].str.lower() == table.lower()]

    select_lines = []
    group_by_lines = []

    for _, row in table_df.iterrows():
        col = row['column_names']
        line = f"trim(tbl.{col}) as {col}"

        group_line = f"trim({col})"
        select_lines.append(line)
        group_by_lines.append(group_line)

    select_block = ",\n  ".join(select_lines)
    group_by_block = ",\n  ".join(group_by_lines)

    view_sql = f"""CREATE OR REPLACE VIEW silver.V_{table} AS
SELECT
  {select_block}
FROM bronze.{table} tbl
GROUP BY
  {group_by_block};
"""
    print(view_sql)

    full_sql+=view_sql

CREATE OR REPLACE VIEW silver.V_tbl_clcl_claim AS
SELECT
  trim(tbl.AGAG_ID) as AGAG_ID,
  trim(tbl.ATXR_SOURCE_ID) as ATXR_SOURCE_ID,
  trim(tbl.CLCL_ACD_AMT) as CLCL_ACD_AMT,
  trim(tbl.CLCL_ACD_DT) as CLCL_ACD_DT,
  trim(tbl.CLCL_ACD_IND) as CLCL_ACD_IND,
  trim(tbl.CLCL_ACD_STATE) as CLCL_ACD_STATE,
  trim(tbl.CLCL_ACPT_DTM) as CLCL_ACPT_DTM,
  trim(tbl.CLCL_AG_SOURCE) as CLCL_AG_SOURCE,
  trim(tbl.CLCL_AIAI_EOB_IND) as CLCL_AIAI_EOB_IND,
  trim(tbl.CLCL_BATCH_ACTION) as CLCL_BATCH_ACTION,
  trim(tbl.CLCL_BATCH_ID) as CLCL_BATCH_ID,
  trim(tbl.CLCL_CAP_IND) as CLCL_CAP_IND,
  trim(tbl.CLCL_CE_IND) as CLCL_CE_IND,
  trim(tbl.CLCL_CL_SUB_TYPE) as CLCL_CL_SUB_TYPE,
  trim(tbl.CLCL_CL_TYPE) as CLCL_CL_TYPE,
  trim(tbl.CLCL_COB_EOB_IND) as CLCL_COB_EOB_IND,
  trim(tbl.CLCL_COBRA_IND) as CLCL_COBRA_IND,
  trim(tbl.CLCL_CUR_STS) as CLCL_CUR_STS,
  trim(tbl.CLCL_CURR_ILL_DT) as CLCL_CURR_ILL_DT,
  trim(tbl.CLCL_DRAG_DT) as CLCL_DRAG_DT,
  trim(tbl.CLCL_DRAG_OR_IND) as CLCL_DRAG_OR_IND,
  t

## Saving to file

In [14]:
with open("create_views.sql","w") as f:
    f.write(full_sql)